# IPython Notebook - N-gram Tutorial

I've always wondered how chat bots like [Alice](http://alice.pandorabots.com/) work. Now, they are obviously much more complex than this tutorial will delve into, but we can touch on some of the core principles. One of them is this idea of understanding the relationships between words in sentences. How can we get a machine to understand these relationships?

Turns out there's the right way, and then there's the easy way. The right way involves delving deep into [semantic networks](https://en.wikipedia.org/wiki/Semantic_network) and ontologies, something I'd touched upon in my climate modelling days, but never mind that; We're doing **The Easy Way**.

### The Easy Way

Conversely, the easy way to learn the relationships is by throwing lots of data *en masse* at a machine, and letting it build up a model of the relationships (*this sounds suspiciously like Machine Learning*). 

An even simpler form of that is to track the number of words that are in sequence with one another, and keeping track of the frequency at which this occurs. We're actually starting to describe something that uses [N-grams](https://en.wikipedia.org/wiki/N-gram). An N-gram is a contiguous (*order matters*) sequence of items, which in this case is the words in text.

What we want to do is build up a dictionary of N-grams, which are pairs, triplets or more (*the N*) of words that pop up in the training data, with the value being the number of times they showed up. After we have this dictionary, as a naive example we could actually predict sentences by just randomly choosing words within this dictionary and doing a weighted random sample of the connected words that are part of n-grams within the keys.

Lets see how far we can get with N-grams without outside resources.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load data
path = "./../data/harry_potter_3.txt"
txt = open(path).read().lower()
print('corpus length:', len(txt))

('corpus length:', 626260)


Great, now lets split into words into a big list, splitting on anything non-alphanumeric [A-Za-z0-9] (as well as punctuation) and forcing everything lowercase

In [3]:
import re
words = re.split('[^A-Za-z]+', txt.lower())
words = filter(None, words) # Remove empty strings

# Print length of list
print len(words)



108805


## Sets
From this we can now generate N-grams, lets start with a 1-gram, basically the set of all the words

    *Note* : One could use a dictionary instead of a set and keeping count of the occurances gives word frequency

In [4]:
#import sets

# Create set of all unique words, this throws away any information about frequency however
gram1 = set(words)

print len(gram1)

# Instead of printing all the elements in the set, create an iterator and print 20 elements only
gram1_iter = iter(gram1)
print [gram1_iter.next() for i in xrange(20)]

7393
['raining', 'foul', 'four', 'woods', 'clotted', 'spiders', 'hanging', 'conjuring', 'conjure', 'sevens', 'increase', 'unblocked', 'electricity', 'wizardry', 'formless', 'wholeheartedly', 'sputter', 'lord', 'flicking', 'sinking']


Lets try and get the 2-gram now, which is pairs of words. Let's have a quick look to see the last 10 and how they look.

In [5]:
# See the last 10 pairs
for i in xrange(len(words)-10, len(words)-1):
    print words[i], words[i+1]

like a
a much
much better
better summer
summer than
than the
the last
last the
the end


Okay, seems good, lets get all word pairs, and then generate a set of unique pairs from it

In [6]:
word_pairs = [(words[i], words[i+1]) for i in xrange(len(words)-1)]
print len(word_pairs)

gram2 = set(word_pairs)
print len(gram2)

# Print 20 elements from gram2
gram2_iter = iter(gram2)
print [gram2_iter.next() for i in xrange(20)]

108804
53720
[('playing', 'as'), ('tonic', 'was'), ('my', 'final'), ('staring', 'along'), ('of', 'terrified'), ('three', 'minutes'), ('yeh', 'ron'), ('droppin', 'me'), ('not', 'listening'), ('it', 'looks'), ('have', 'made'), ('her', 'job'), ('swishing', 'noise'), ('knuckles', 'of'), ('harry', 'his'), ('fist', 'petunia'), ('do', 'beg'), ('want', 'you'), ('he', 'entered'), ('be', 'fit')]


## Frequency

Okay, that was fun, but this isn't enough, we need frequency if we want to have any sense of probabilities, which is what N-grams are about. Instead of using sets, lets create a dictionary with counts

In [7]:
gram1 = dict()

# Populate 1-gram dictionary
for word in words:
    if gram1.has_key(word):
        gram1[word] += 1
    else:
        gram1[word] = 1 # Start a new entry with 1 count since saw it for the first time.

# Turn into a list of (word, count) sorted by count from most to least
gram1 = sorted(gram1.items(), key=lambda (word, count): -count)

# Print top 20 most frequent words
print gram1[:20]

[('the', 4991), ('and', 2620), ('to', 2554), ('a', 2111), ('he', 2065), ('of', 2030), ('harry', 2005), ('was', 1649), ('s', 1545), ('his', 1470), ('said', 1436), ('it', 1398), ('you', 1388), ('i', 1304), ('in', 1201), ('had', 888), ('t', 880), ('that', 862), ('at', 801), ('ron', 756)]


For Pride and Prejudice, the words 'the', 'to', 'of', and 'and' were the top four most common words. Sounds about right, not too interesting yet, lets see what happens with 2-grams.

In [8]:
gram2 = dict()

# Populate 2-gram dictionary
for i in xrange(len(words)-1):
    key = (words[i], words[i+1])
    if gram2.has_key(key):
        gram2[key] += 1
    else:
        gram2[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram2 = sorted(gram2.items(), key=lambda (_, count): -count)

# Print top 20 most frequent words
print gram2[:20]

[(('of', 'the'), 461), (('in', 'the'), 292), (('said', 'harry'), 250), (('to', 'the'), 249), (('he', 'was'), 234), (('out', 'of'), 230), (('on', 'the'), 227), (('it', 'was'), 226), (('at', 'the'), 219), (('didn', 't'), 200), (('harry', 's'), 180), (('don', 't'), 171), (('said', 'ron'), 170), (('and', 'hermione'), 162), (('it', 's'), 156), (('he', 'had'), 154), (('into', 'the'), 149), (('he', 'said'), 148), (('was', 'a'), 147), (('in', 'a'), 140)]


It looks like `"of the"` and `"to be"` are the top two most common 2-grams, sounds good. 

## Next word prediction
What can we do with this? Well lets see what happens if we take a random word from all the words, and build a sentence by just choosing the most common pair that has that word as it's start.

In [9]:
start_word = words[len(words)/4]
print start_word

boarhound


I just went ahead and chose the word that appears $1/4$ of the way into words, random enough.

Now in a loop, iterate through the frequency list (most frequent first) and see if it matches the first word in a pair, if so, the next word is the second element in the word pair, and continue with that word. Stop after N words or the list does not contain that word.

    *Note* : gram2 is a list that contains (key,value) where key is a word pair (first, second),
             so you need element[0][0] for first word and element [0][1] for second word

In [10]:
def get2GramSentence(word, n = 50):
    for i in xrange(n):
        print word,
        # Find Next word
        word = next((element[0][1] for element in gram2 if element[0][0] == word), None)
        if not word:
            break

word = start_word
print "Start word: %s" % word

print "2-gram sentence: \"",
get2GramSentence(word, 20)
print "\""

Start word: boarhound
2-gram sentence: " boarhound at the door of the door of the door of the door of the door of the door of "


It gets stuck in a loop pretty much straight away. Not very interesting, try out other words and see what happens.

In [11]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "Start word: %s" % word

    print "2-gram sentence: \"",
    get2GramSentence(word, 20)
    print "\""

Start word: and
2-gram sentence: " and hermione s a very good bye harry s a very good bye harry s a very good bye harry "
Start word: he
2-gram sentence: " he was a very good bye harry s a very good bye harry s a very good bye harry s "
Start word: she
2-gram sentence: " she was a very good bye harry s a very good bye harry s a very good bye harry s "
Start word: when
2-gram sentence: " when he was a very good bye harry s a very good bye harry s a very good bye harry "
Start word: john
2-gram sentence: " john "
Start word: never
2-gram sentence: " never seen the door of the door of the door of the door of the door of the door of "
Start word: i
2-gram sentence: " i m not to the door of the door of the door of the door of the door of the "
Start word: how
2-gram sentence: " how could see you know what s a very good bye harry s a very good bye harry s a "


## Weighted random choice based on frequency
Same thing. Okay, lets randomly choose from the subset of all 2grams that matches the first word, using a weighted-probability based on counts.

In [12]:
import random
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w > r:
         return c
      upto += w
    
def get2GramSentenceRandom(word, n = 50):
    for i in xrange(n):
        print word,
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram2 if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]

In [13]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "Start word: %s" % word

    print "2-gram sentence: \"",
    get2GramSentenceRandom(word, 20)
    print "\""

Start word: and
2-gram sentence: " and peter you will not got down on george looked as fast but what do i he raised his head "
Start word: he
2-gram sentence: " he s eyes with dusty classroom with his enormous wings fell silent tears ron harry knew what kept to practice "
Start word: she
2-gram sentence: " she said neville regularly went haywire just one that moment please ron says i have found themselves outside the thud "
Start word: when
2-gram sentence: " when we ll be having your books it high over by the ruined painting of explosion grunted ernie mcmillan told "
Start word: john
2-gram sentence: " john "
Start word: never
2-gram sentence: " never speak to punish you complete confidence in a little horns had seen him his teachers found it s reach "
Start word: i
2-gram sentence: " i m coming back to it was walking very confused impression that going to sleep even started again you can "
Start word: how
2-gram sentence: " how dare you re helping himself free hand had an you know th

**Now that's way more interesting!** Those are starting to look like sentences!

    *Note* It's pretty interesting to see that for the sentence " when he believed him from the amiable but mrs hurst s being ill of being the discussion of course of ", we have hurst s, which we can tell came from Hurst's, an artifact of our stripping away all punctuation but keeping the s.

Let's try a longer sentence

In [14]:
word = 'it'
print "Start word: %s" % word
print "2-gram sentence: \"",
get2GramSentenceRandom(word, 50)
print "\""

Start word: it
2-gram sentence: " it so s shrivelfig as though he said hippogriffs attack anyone can explain to him but you could have put herrmone s said harry clutched her precious dogs they lose a lot of us dinner what we ll ever seen that filled with you too said professor trelawney looked over "


Pretty cool, lets see what happens when we go to N-grams above 2.
## Tri-grams and more
Okay, let's create a Ngram generator that can let us make ngrams of arbitrary sizes

In [15]:
def generateNgram(n=1):
    gram = dict()
    
    # Some helpers to keep us crashing the PC for now
    assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in xrange(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if gram.has_key(key):
            gram[key] += 1
        else:
            gram[key] = 1

    # Turn into a list of (word, count) sorted by count from most to least
    gram = sorted(gram.items(), key=lambda (_, count): -count)
    return gram

trigram = generateNgram(3)
# Print top 20 most frequent ngrams
print trigram[:20]

[(('out', 'of', 'the'), 96), (('ron', 'and', 'hermione'), 92), (('there', 'was', 'a'), 81), (('i', 'don', 't'), 66), (('in', 'front', 'of'), 45), (('the', 'rest', 'of'), 44), (('harry', 'and', 'hermione'), 42), (('he', 'didn', 't'), 41), (('harry', 'ron', 'and'), 40), (('as', 'though', 'he'), 39), (('harry', 'and', 'ron'), 39), (('the', 'end', 'of'), 38), (('rest', 'of', 'the'), 35), (('harry', 'didn', 't'), 32), (('one', 'of', 'the'), 31), (('was', 'going', 'to'), 31), (('seemed', 'to', 'be'), 29), (('crabbe', 'and', 'goyle'), 28), (('fred', 'and', 'george'), 28), (('said', 'professor', 'mcgonagall'), 27)]


Cool! Okay, let's see a selection of sentences for N-grams with N = 2 to 10 and a few starting words!

In [16]:
def getNGramSentenceRandom(gram, word, n = 50):
    for i in xrange(n):
        print word,
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
for n in xrange(2,10):
    # Generate ngram list
    print
    print "Generating %d-gram list..." % n,
    ngram = generateNgram(n)
    print "Done"
    
    # Try out a bunch of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
        print "  %d-gram: \"" % n,
        getNGramSentenceRandom(ngram, word, 15)
        print "\""


Generating 2-gram list... Done
  2-gram: " and something shiny badge to see said madam pomfrey can get up his bedspread ron "
  2-gram: " he pleased as hermione we doing nothing i did he was that uncle vernon you "
  2-gram: " she had been thirteen people reckon filch restore her seat right hand and his eyes "
  2-gram: " when wizards put on weekends that made his stool and pointed nose out of women "
  2-gram: " john "
  2-gram: " never hurt a stone slide then lifted there is it s face was in britain "
  2-gram: " i haven t be back so that the window and adding your acceleration and hermione "
  2-gram: " how they had been wearing canary yellow eyes said professor snape doesn t you actualy "

Generating 3-gram list... Done
  3-gram: " and felt just can get any chocolate terrible terrible fate terrible thing so harry saw "
  3-gram: " he said dumbledore to that he said neville was listed as they haven t got "
  3-gram: " she s something in honeydukes owners once and ron said hagrid to

You can clearly see the sentences getting better and better with larger n-grams, this correlates to the ngram having more foresight into the sentence structure.

In [17]:
# Generate 10gram list
print
print "Generating %d-gram list..." % n,
gram10 = generateNgram(10)
print "Done"


Generating 9-gram list... Done


Let's play with the 10gram and see what sort of sentence comes out.

In [18]:
# Try out a bunch of sentences
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "  %d-gram: \"" % n,
    getNGramSentenceRandom(ngram, word, 100)
    print "\""

  9-gram: " and they obviously not being louder and hermione kept looking good enough to sniff hopefully it before dumbledore had just just drop he did he was the castle his ink self anymore fact that neville s desk his feet if he d done for him for a fresh attempt to me to harry knew the tiny owl of talent on the now smiling i could they set out of the door it happen when you what s head boy i want to seem to buzz excitedly keep a cackling harry saw you are in called stan old socks well the "
  9-gram: " he was under his head was filled the disposal o you think about it said harry had finally winning the excitement something of his normally ruddy form when he held him to use it waving his nose smashed into diagon alley and mrs weasley made a stick and stared at harry there that no weeping hagrid had to provoke him and began to the crowd and he seemed to the look of the fire still found excuses to fulfill even himself standing up an enormous surprise hermione p p p s invisibility cloak

Looks almost like normal sentences if you squint a little! Well, that was fun. Next up let's see some ways to improve upon this.

Instead of just taking the next word every time, we could take the next k words etc.

To be continue...

In [19]:
# Generate 10gram list
n = 50
print
print "Generating %d-gram list..." % n,
gram30 = generateNgram(n)
print "Done"


Generating 50-gram list... Done


In [20]:
# Try out a bunch of sentences
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "  %d-gram: \"" % n,
    getNGramSentenceRandom(ngram, word, 20)
    print "\""

  50-gram: " and harry as a tent a danger severus i m coming from father but black said harry was the other "
  50-gram: " he tried to hogwarts house behind him until last night i m pickling some practice for lunch though he leaned "
  50-gram: " she was sitting room past dealings with buckbeak he really want to the werewolf out where i could hurt it "
  50-gram: " when harry said you ll give up into their front knees and the prisoners in the one can t have "
  50-gram: " john "
  50-gram: " never trust has he was eighteen moony wormtail padfoot peter he did not going to persuade her throat no professor "
  50-gram: " i lost her said percy and seeker in your own hedwig and the shops were the news of it meanwhile "
  50-gram: " how nice bludger ducking beneath beds of fur suddenly stood glaring at the day me said harry roared at the "
